# U-Net Workshop

Here is an example U-Net implementation using minified data based upon creative commons dataset available at https://wiki.cancerimagingarchive.net/display/Public/HNSCC.

In [ ]:
# Copyright 2021 Radiotherapy AI Pty Ltd

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#    http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import pathlib
import random
import shutil
import urllib.request

import imageio
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
BATCH_SIZE = 512
GRID_SIZE = 64

COLOURS_AND_LABELS = [
    ("#ff7f0e", "left parotid"),
    ("#2ca02c", "right parotid"),
    ("#d62728", "external"),
]
NUM_CONTOURS = len(COLOURS_AND_LABELS)

## Download the data

In [ ]:
zip_url = "https://github.com/RadiotherapyAI/unet-workshop/releases/download/mini-parotid/mini-parotid.zip"
zip_filepath = "data.zip"

data_directory = pathlib.Path("data")

if not data_directory.exists():
    urllib.request.urlretrieve(zip_url, zip_filepath)
    shutil.unpack_archive(zip_filepath, data_directory)

In [ ]:
# Investigate the `data_directory` with glob and pathlib

## Build the TensorFlow pipeline and investigate the data

In [ ]:
def get_path_pairs(dataset_type):
    training_image_paths = list((data_directory / dataset_type).glob("*/*.image.png"))
    training_mask_paths = [
        path.parent / f"{path.name.split('.')[0]}.masks.png"
        for path in training_image_paths
    ]

    path_pairs = [
        (str(image), str(mask))
        for image, mask in zip(training_image_paths, training_mask_paths)
    ]

    return sorted(path_pairs)

In [ ]:
@tf.function
def load(path_pair):
    image_path = path_pair[0]
    masks_path = path_pair[1]
    
    image_raw = tf.io.read_file(image_path)
    image = tf.io.decode_image(image_raw, channels=1, dtype=tf.uint8)

    masks_raw = tf.io.read_file(masks_path)
    masks = tf.io.decode_image(masks_raw, channels=3, dtype=tf.uint8)

    return image / 255, masks / 255

In [ ]:
def create_datasets(dataset_type):
    path_pairs = get_path_pairs(dataset_type)
    dataset = tf.data.Dataset.from_tensor_slices(path_pairs)
    dataset = dataset.shuffle(len(path_pairs), reshuffle_each_iteration=True)
    dataset = dataset.map(load)

    batched_dataset = dataset.batch(BATCH_SIZE)
    batched_dataset = batched_dataset.prefetch(tf.data.AUTOTUNE)

    return dataset, batched_dataset

In [ ]:
# Use the `create_datasets` function to create a set of training and validation datasets

In [ ]:
# Extract an image and masks out of the validation dataset

In [ ]:
# Use matplotlib to investigate the extracted image and masks

In [ ]:
def plot_contours(ax, image, masks):
    ax.imshow(image[:, :, 0], cmap="gray")

    for i, (colour, label) in enumerate(COLOURS_AND_LABELS):
        c = ax.contour(masks[..., i], colors=[colour], levels=[0.5])
        c.collections[0].set_label(label)

    ax.axis("equal")
    ax.legend()

In [ ]:
# Use the `plot_contours` function to view the masks as contours on the ct image

## Create the 2D U-Net model

In [ ]:
def activation(x):
    x = tf.keras.layers.Activation("relu")(x)

    return x


def convolution(x, number_of_filters, kernel_size=3):
    x = tf.keras.layers.Conv2D(
        number_of_filters, kernel_size, padding="same", kernel_initializer="he_normal"
    )(x)

    return x


def conv_transpose(x, number_of_filters, kernel_size=3):
    x = tf.keras.layers.Conv2DTranspose(
        number_of_filters,
        kernel_size,
        strides=2,
        padding="same",
        kernel_initializer="he_normal",
    )(x)

    return x


def encode(
    x,
    number_of_filters,
    number_of_convolutions=2,
):
    for _ in range(number_of_convolutions):
        x = convolution(x, number_of_filters)
        x = activation(x)
    skip = x

    x = tf.keras.layers.MaxPool2D()(x)
    x = activation(x)

    return x, skip


def decode(
    x,
    skip,
    number_of_filters,
    number_of_convolutions=2,
):
    x = conv_transpose(x, number_of_filters)
    x = activation(x)

    x = tf.keras.layers.concatenate([skip, x], axis=-1)

    for _ in range(number_of_convolutions):
        x = convolution(x, number_of_filters)
        x = activation(x)

    return x

In [ ]:
inputs = tf.keras.layers.Input((GRID_SIZE, GRID_SIZE, 1))

x = inputs
skips = []

In [ ]:
# Create the encoding layers and collect the skips

In [ ]:
# Create the decoding layers with skips included

In [ ]:
x = tf.keras.layers.Conv2D(
    NUM_CONTOURS,
    1,
    activation="sigmoid",
    padding="same",
    kernel_initializer="he_normal",
)(x)

model = tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
# Display a summary of the model

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.Precision(),
    ],
)

In [ ]:
# Use the newly created model to create a prediction

In [ ]:
def plot_with_prediction(image, masks, pred_masks):
    fig, ax = plt.subplots(figsize=(12, 6), ncols=2)
    plot_contours(ax[0], image, masks)
    plot_contours(ax[1], image, pred_masks)

In [ ]:
# Use the `plot_with_prediction` function to plot the prediction result

## Training

Now that we've set up our data pipeline, and structured our model, let's train it.

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        pred_masks = model.predict(image[None, ...])[0, ...]
        plot_with_prediction(image, masks, pred_masks)

        plt.show()
        print("\nSample Prediction after epoch {}\n".format(epoch + 1))

In [ ]:
# Use `model.fit` to run the training

In [ ]:
# Plot the history of the training and validation loss